<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Personality%20Enabled%20MF/PEMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PEMF MODEL

In [ ]:
# importing packages
import numpy as np
import numba as nb
import scipy.sparse as sp
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import scipy.spatial as spt
import statistics
import math
import json
from sklearn.model_selection import GridSearchCV
from random import sample

In [ ]:
# importing packages
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.MENTION,p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from textblob import Word
import re
punctuation = re.compile(r'[-.?&!,:;()|0-9]')

In [ ]:
def pemf(R,user_list,item_list,S,n,m,K,alpha,lambdaU,lambdaI,lambdaS,R_test,user_list_test,item_list_test,S_test):
    def sigmoid(z):
        return 1 + ((5.0 - 1.0) / (1+np.exp(-z)))
    def dsigmoid(z):
        return (5.0 - 1.0) * np.exp(-z)/np.power((1+np.exp(-z)),2)
    def rmse(U,I,R):
        keylist = []
        dok_keys = np.array(R.todok().keys()).T.reshape([1,1])[0][0]
        for k in dok_keys:
          row = []
          for l in k:
            row.append(l)
          keylist.append(row) 
        keylist = np.array(keylist)
        utl = keylist[:, 0]
        itl = keylist[:, 1]
        error = (get_csrmat(sigmoid(U.dot(I.T)),utl,itl)-R).power(2).sum()/R.nnz
        error = np.sqrt((get_csrmat(sigmoid(U.dot(I.T)),utl,itl)-R).power(2)).sum()/np.sqrt(R.nnz)
        return np.sqrt(error)
    def mae(U,I,R):
        keylist = []
        dok_keys = np.array(R.todok().keys()).T.reshape([1,1])[0][0]
        for k in dok_keys:
          row = []
          for l in k:
            row.append(l)
          keylist.append(row) 
        keylist = np.array(keylist)
        utl = keylist[:, 0]
        itl = keylist[:, 1]
        error = abs(get_csrmat(sigmoid(U.dot(I.T)),utl,itl)-R).sum()/R.nnz 
        return error
    def get_csrmat(mat,user_list,item_list):
        indx = user_list*mat.shape[1]+item_list
        return sp.csr_matrix((np.take(np.array(mat),indx),(user_list,item_list)),shape=(mat.shape[0],mat.shape[1]))
    def costL(U,I,S,user_list,item_list):
        tmp = U.dot(I.T)
        Rx = get_csrmat(sigmoid(tmp),user_list,item_list)
        cost = 0.5*((R - Rx).power(2)).sum()+0.5*lambdaU*np.linalg.norm(U)**2+0.5*lambdaI*np.linalg.norm(I)**2
        cost += 0.5*lambdaS*np.power(U-S.dot(U),2).sum()
        return cost
    def gradient(U,I,S,user_list,item_list):
        dU = np.zeros(U.shape)
        dI = np.zeros(I.shape)
        dU = lambdaU*U
        tmp = U.dot(I.T)
        Rv = get_csrmat(dsigmoid(tmp),user_list,item_list)
        Rx = get_csrmat(sigmoid(tmp),user_list,item_list)
        dU += Rv.multiply((Rx-R)).dot(I)
        dU += lambdaS*(U-S.dot(U))-lambdaS*S.T.dot((U-S.dot(U)))
        dI = lambdaI*I
        dI += (Rv.multiply((Rx-R))).T.dot(U)
        if np.max(dU)>1:
            dU = dU/np.max(dU)
        if np.max(dI)>1:
            dI = dI/np.max(dI)
        return dU,dI


    def train(U,I,S,user_list,item_list):
        res=[]
        steps=10
        learning_rate = alpha
        pregradU = 0
        pregradI = 0
        tol=1e-6
        momentum = 0.9
        stage = max(steps/100 , 1)
        for step in range(steps):
            start = time.time()
            dU,dI = gradient(U,I,S,user_list,item_list)
            dU = dU + momentum*pregradU
            dI = dI + momentum*pregradI
            pregradU = dU
            pregradI = dI
            if not step%stage and rate>0.001:
                rate = 0.95*rate
            U -= rate * dU
            I -= rate * dI
            e = costL(U,I,S,user_list,item_list) / (len(U) * len(I))
            res.append(e)
            if not step%stage:
                #print("step",step,"error", e, "time", time.time() - start)
                #print("RMSE",rmse(U,V,R), "MAE",mae(U,V,R))
                e1 = e
            if step>150:# or abs(sum(res[-3:])-sum(res[-13:-10]))<tol:
                #print("====================")
                #print("stop in %d step"%(step))
                #print("error is ",e)
                #print("====================")
                break
        return U, I


    U = np.random.normal(0,0.01,size=(R.shape[0],k))
    I = np.random.normal(0,0.01,size=(R.shape[1],k))
    Utest = np.random.normal(0,0.01,size=(R_test.shape[0],k))
    Itest = np.random.normal(0,0.01,size=(R_test.shape[1],k))
    start = time.time()
    U,I = train(U,I,S,user_list,item_list)

    #print("=================RESULT=======================")
    #print('K:%d,lambdaU:%s, lambdaI:%s,lambdaS:%s' \
    #        %(K,lambdaU,lambdaI,lambdaS))
    #print("mae",mae(U,V,R_test))
    #print("time",time.time() - start)
    #print("========================================")
    #print("mae recal", mean_absolute_error((U@V.T).toarray(), R_test))

    print("rmse ",rmse(Utest,Itest,R_test), "mae ",mae(Utest,Itest,R_test), "time",time.time() - start)
    return 0

In [ ]:
def calc_distance(A,A1):
  similarity = np.dot(A, A1.T)
  square_mag = np.diag(similarity)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = similarity * inv_mag
  cosine = cosine.T * inv_mag
  return cosine

# target

## data

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})
arr = tr[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating']]

In [ ]:
# amazon magazines personality csv
path1 = r'/content/drive/MyDrive/Per_CD_RS/magazine_output_final.csv'
with open(path1, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile, nrows=100)
#arr = np.array(df[['0', '1', '2', '3', '4']])

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile)
#az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
t = pd.merge(df, az, on='userId', how='inner')
arr = t[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating']]
t.shape

## matrix factorization

In [ ]:
src = tr[['userId', 'itemId', 'rating']]
arr = pd.DataFrame(arr)
arr['userId'] = az['userId']
arr['itemId'] = np.array(az['itemId'])
arr['rating'] = az['rating']

In [ ]:
# Construction of pivot matrix of ratings ru with respect to users u
ru = arr.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
ru_m = ru > 0
ru_m = ru_m.replace(True, 1)
ru_m = ru_m.replace(False, 0)
ru = np.array(ru)
ru_m = np.array(ru_m)

In [ ]:
ru = arr.groupby(['userId', 'itemId'])['rating'].sum().unstack()
ru = pd.DataFrame(ru)
ru.shape

In [ ]:
# dividing into train and test of rating matrix ru

def split_data(ru, s, k):
  split = s
  k = k
  test = []
  set1  = ru.iloc[: , :int(ru.shape[1]/split)]
  set1 = set1.replace(np.nan, 0)
  for i in range(k):
    set2 = ru.iloc[: , i*int(ru.shape[1]/split):(i+1)*int(ru.shape[1]/split)]
    set2 = set2.replace(np.nan, 0)
    set1 = pd.concat([set1, set2], axis=1)
    set1  = ru.iloc[: , :int(ru.shape[1]/split)]
    set1 = set1.replace(np.nan, 0)
    set2  = ru.iloc[: , int(ru.shape[1]/split): 2*int(ru.shape[1]/split)]
    set2 = set2.replace(np.nan, 0)
    xx = pd.concat([set1, set2], axis=1)
    test.append(xx)
  return test

In [ ]:
# with only simpers and simrate use this

def tgt_pred(a, k, b, lp, ls, g, fold, split):

    train_size = split
    lr, k, beta, lambda_u, lambda_s, thr = a, k, b, lp, ls, g
    df_copy = arr.copy()
    train_set = df_copy.sample(frac=train_size).reset_index()
    user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
    #test_set = df_copy.drop(train_set.index).reset_index()
    test_set = pd.DataFrame(split_data(df_copy, (1-train_size)*10, fold)[0])
    user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))

    df = pd.DataFrame(user_features_train) 
    df.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
    df['userId'] = train_set.userId
    df2 = df.groupby(by='userId').mean().reset_index()
    df3 = np.array(df2[["open", 'cons', 'extra', 'agree', 'neuro']])
    

    
    ru = train_set.groupby(['userId', 'itemId'])['rating'].sum().unstack()
    ru = pd.DataFrame(ru)
    set1 = ru.iloc[: , :int(ru.shape[1]/(1-train_size)*10)]
    set1 = set1.replace(np.nan, 0)
    #for i in range(1,2):
    # set2 = ru.iloc[: , i*int(ru.shape[1]/10):(i+1)*int(ru.shape[1]/10)]
    #  set2 = set2.replace(np.nan, 0)
    #  set1 = pd.concat([set1, set2], axis=1)
    ru = set1
    ru_m = ru > 0
    ru_m = ru_m.replace(True, 1)
    ru_m = ru_m.replace(False, 0)
    ru = np.array(ru)
    ru_m = np.array(ru_m)

    R0 = ru    # rating matrix
    max_r = 5.0    # max _rating   
    N,M= ru.shape[0],ru.shape[1]     # # n : no of users   m : no of items 
    K = k                       #k : latent dimension        (mx10)@(10xn)
    lambdaU,lambdaI,lambdaS= lambda_u, lambda_u, lambda_a
    keys = []

    R=sp.dok_matrix((N,M))   # create sparse matrix for user x item
    T=sp.dok_matrix((N,N))   # create sparse matrix for trust among users user x user
    for i in range(len(R0)):    # no of users
        for j in range(len(R0[i])):   # no of items
            if R0[i][j]>0:    # if rating is present   
                keys.append([i,j])
                R[i,j] = R0[i][j]     

    dft = pd.DataFrame(user_features_test) 
    dft.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
    dft['userId'] = test_set.userId
    df2t = dft.groupby(by='userId').mean().reset_index()
    df3t = np.array(df2t[["open", 'cons', 'extra', 'agree', 'neuro']])

    rut = test_set.pivot_table(index='userId',columns='itemId',values='rating')
    rut = test_set.groupby(['userId', 'itemId'])['rating'].sum().unstack()
    rut = pd.DataFrame(rut)
    rut = rut.fillna(0)
    ru_mt = rut > 0
    ru_mt = ru_mt.replace(True, 1)
    ru_mt = ru_mt.replace(False, 0)
    rut = np.array(rut)
    ru_mt = np.array(ru_mt)

    R0t = rut    # rating matrix
    max_rt = 5.0    # max _rating   
    Nt,Mt= rut.shape[0],rut.shape[1]     # # n : no of users   m : no of items 
    K = k                        #k : latent dimension        (mxk)@(kxn)
    lambdaU,lambdaI,lambdaS = lambda_u, lambda_u, lambda_s
    keyst = []

    Rt = sp.dok_matrix((Nt,Mt))   # create sparse matrix for user x item
    Tt = sp.dok_matrix((Nt,Nt))   # create sparse matrix for user x user
    for i in range(len(R0t)):    # no of users
        for j in range(len(R0t[i])):   # no of items
            if R0t[i][j]>0:    # if rating is present   
                keyst.append([i,j])
                Rt[i,j] = R0t[i][j]      

    
    # get trust factor between users trust_uv(user x user)  and sim_uv(user x user) t_uv(user x user)


    simpers_uv = 1 - calc_distance(df3, df3)
    simpers_uv = np.nan_to_num(trust_uv)
    simpers_uv_s = simpers_uv.copy()
    simpers_uv = pd.DataFrame(simpers_uv >= thr)
    simpers_uv = simpers_uv.replace(True, 1)
    simpers_uv = simpers_uv.replace(False, 0)
    simpers_uv = np.array(simpers_uv)

    neighbours = []
    for j in range(len(trust_uv)):
      row = []
      for m in range(len(trust_uv[j])):
        if trust_uv[j][m] == 1:
          row.append(m)
      neighbours.append(row)
    neighbours = np.array(neighbours)
    simrate_uv = 1- calc_distance(ru, ru)
    simrate_uv = np.nan_to_num(simrate_uv)
    #ru_tf2 = ru_tf[0:trust_uv.shape[0]]
    #ru_tf2 = ru_tf[:,:len(trust_uv)]
    s_uv = np.add(beta * simpers_uv_s, (1-beta)* simrate_uv)
    #s_uv = np.multiply(np.add(beta * trust_uv_s, (1-beta)* sim_uv), ru_tf)

    max_value = max(s_uv.flatten())
    min_value = min(s_uv.flatten())
    S_df = pd.DataFrame(s_uv)
    S_df = S_df >=  np.subtract(S_df, min_value) /(max_value - min_value)   # thr is threshold value
    S_df = S_df.replace(True, 1)
    S_df = S_df.replace(False, 0)
    S_df = np.array(S_df)
    S = sp.csr_matrix(S_df)   # fill up trust matrix from user realtionships   *******


    simpers_uv_t = 1 - calc_distance(df3t, df3t)
    simpers_uv_t = np.nan_to_num(simpers_uv_t)
    simpers_uv_st = simpers_uv_t.copy()
    simpers_uv_t = pd.DataFrame(simpers_uv_t >= thr)
    simpers_uv_t = simpers_uv_t.replace(True, 1)
    simpers_uv_t = simpers_uv_t.replace(False, 0)
    simpers_uv_t = np.array(simpers_uv_t)

    neighbours_t = []
    for j in range(len(simpers_uv_t)):
      row = []
      for m in range(len(trust_uv_t[j])):
        if trust_uv_t[j][m] == 1:
          row.append(m)
      neighbours_t.append(row)
    neighbours_t = np.array(neighbours_t)
    simrate_uv_t = 1- calc_distance(rut, rut)
    simrate_uv_t = np.nan_to_num(simrate_uv_t)
    #ru_tf2 = ru_tf[0:trust_uv.shape[0]]
    #ru_tf2 = ru_tf[:,:len(trust_uv)]
    s_uv_t = np.add(beta * simpers_uv_st, (1-beta)* simrate_uv_t)
    #t_uv = np.multiply(np.add(beta * trust_uv_s, (1-beta)* sim_uv), ru_tf)

    max_value = max(s_uv_t.flatten())
    min_value = min(s_uv_t.flatten())
    S_df_t = pd.DataFrame(s_uv_t)
    S_df_t = S_df_t >=  np.subtract(S_df_t, min_value) /(max_value - min_value)   # thr is threshold value
    S_df_t = S_df_t.replace(True, 1)
    S_df_t = S_df_t.replace(False, 0)
    S_df_t = np.array(S_df_t)
    St = sp.csr_matrix(S_df_t)   # fill up trust matrix from user realtionships   *******


    
    keys = np.array(keys)        # get the keys
    keyst = np.array(keyst)
    R,S = R.tocsr(),S.tocsr()
    Rt,St = Rt.tocsr(), St.tocsr()
    pemf(R,keys[:, 0],keys[:, 1],T,N,M,K,lambdaU,lambdaI,lambdaS,Rt,keyst[:, 0],keyst[:, 1],St)

## Run

In [ ]:
alpha = [0.0001]
K = [40]
beta = [0.5]
lambda_u = [0.02]
lambda_s = [0.5]
gamma = [0.7]
fold = 1
split = 0.8

In [ ]:
a, k, b, lp, ls, g = 0.0001, 20, 0.5, 0.03, 0.8, 0.8

In [ ]:
if __name__ == "__main__":
  for i in range(fold):
    for a in alpha:
      for k in K:
        for b in beta:
          for lp in lambda_u:
            for ls in lambda_s:
              for g in gamma:
                print(a, k, b, lp, ls, g)
                model = tgt_pred(a, k, b, lp, ls, g, fold, split)

0.0001 40 0.5 0.02 0.5 0.7
rmse  7.51462790680989 mae  2.8779589644793466 time 0.06507492065429688


In [ ]:
# @params: 
# trust threshold (0.7) g: gamma ***
# mixing factor (0.5) b: beta  ***
# lambdaU,lambdaI,lambdaS (0.02, 0.02, 0.1)
# no of iterations (100) +++
# no of latent features K (10) ***
# train test split  
# learning rate a: alpha

# metrics
# MAE
# RMSE


# variations
# numerical personality values (0,1)
# binary personality values [0 and 1]
# with demography dbscan clustering
  # ensemble through average voting
  # ensemble through plurality voting 
# without clustering